In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

print("Libraries loaded successfully!")

Libraries loaded successfully!


In [3]:
# Parameters
IMAGE_SIZE = 128  # Resize all images to 128x128
DATASET_PATH = 'data'  # Path to your dataset

# Prepare the dataset
def load_data(dataset_path):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_path))  # Folder names become class labels
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))  # Resize
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels), class_names

# Load and preprocess
images, labels, class_names = load_data(DATASET_PATH)
images = images / 255.0  # Normalize pixel values
labels = to_categorical(labels, num_classes=len(class_names))  # One-hot encode labels

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Loaded {len(images)} images belonging to {len(class_names)} classes.")

Loaded 967 images belonging to 10 classes.


In [5]:
# Bulding the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,393,610 (28.20 MB)

 Trainable params: 7,393,610 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#Train the model

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 345ms/step - accuracy: 0.1582 - loss: 3.0157 - val_accuracy: 0.4588 - val_loss: 2.0363
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 328ms/step - accuracy: 0.5026 - loss: 1.7234 - val_accuracy: 0.8505 - val_loss: 0.8776
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 308ms/step - accuracy: 0.8547 - loss: 0.5723 - val_accuracy: 0.9433 - val_loss: 0.2350
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 318ms/step - accuracy: 0.9485 - loss: 0.2424 - val_accuracy: 0.9897 - val_loss: 0.0708
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 317ms/step - accuracy: 0.9944 - loss: 0.0622 - val_accuracy: 0.9897 - val_loss: 0.0476
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.9945 - loss: 0.0382 - val_accuracy: 0.9948 - val_loss: 0.0263
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 311ms/step - accuracy: 0.9948 - loss: 0.0222 - val_accuracy: 0.9897 - val_loss: 0.0297
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.9916 - loss: 0.0294 - val_accuracy: 0

In [7]:
# Test accuracy

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9944 - loss: 0.0216     
Test Accuracy: 98.97%


In [22]:
# saving the model

model.save('dog_classifier.h5')

# Load and test with new images
from tensorflow.keras.models import load_model

model = load_model('dog_classifier.h5')

# Test with a new image
def predict_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    predictions = model.predict(image)
    predicted_class = class_names[np.argmax(predictions)]

     # Print predictions
    print("Predicted Probabilities:", predictions)
    
    return predicted_class

result = predict_image('testdata/ro36.jpg')
print(f"The predicted dog type is: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Predicted Probabilities: [[4.8635351e-03 5.1806156e-02 6.7588524e-05 4.7946465e-01 2.9351791e-02
  1.0019428e-02 2.7097437e-01 2.2773167e-04 1.5322426e-01 4.5536666e-07]]
The predicted dog type is: Dachshund
